In [1]:
import torch
print(torch.cuda.is_available())

False


In [2]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device detected")

Torch version: 2.2.0+cpu
CUDA available: False
CUDA device name: No CUDA device detected


In [3]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.2.0+cpu
CUDA available: False


In [4]:
from torchtext.datasets import UDPOS, CoNLL2000Chunking

In [5]:
# Load the train, validation, and test datasets
train_iter, valid_iter, test_iter = UDPOS(root='.data', split=('train', 'valid', 'test'))

In [6]:
# Look at the first few items in the training set
for i, item in enumerate(train_iter):
    print(item)
    if i == 0:  # Print first 5 items
        break

[['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.'], ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], ['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']]


In [7]:
train_iter

ShardingFilterIterDataPipe

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

pipeline = TokenClassificationPipeline(model=model, tokenizer=tokenizer)
outputs = pipeline("A test example")
print(outputs)


Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'entity': 'DT', 'score': 0.9997243, 'index': 1, 'word': 'A', 'start': 0, 'end': 1}, {'entity': 'NN', 'score': 0.9997472, 'index': 2, 'word': 'test', 'start': 2, 'end': 6}, {'entity': 'NN', 'score': 0.99973196, 'index': 3, 'word': 'example', 'start': 7, 'end': 14}]


In [9]:
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 46


In [10]:
label_map = model.config.id2label

In [11]:
for id, label in label_map.items():
    print(f"ID: {id}, Tag: {label}")

ID: 0, Tag: O
ID: 1, Tag: ``
ID: 2, Tag: ,
ID: 3, Tag: :
ID: 4, Tag: .
ID: 5, Tag: ''
ID: 6, Tag: $
ID: 7, Tag: #
ID: 8, Tag: CC
ID: 9, Tag: CD
ID: 10, Tag: DT
ID: 11, Tag: EX
ID: 12, Tag: FW
ID: 13, Tag: IN
ID: 14, Tag: JJ
ID: 15, Tag: JJR
ID: 16, Tag: JJS
ID: 17, Tag: -LRB-
ID: 18, Tag: LS
ID: 19, Tag: MD
ID: 20, Tag: NN
ID: 21, Tag: NNP
ID: 22, Tag: NNPS
ID: 23, Tag: NNS
ID: 24, Tag: PDT
ID: 25, Tag: POS
ID: 26, Tag: PRP
ID: 27, Tag: PRP$
ID: 28, Tag: RB
ID: 29, Tag: RBR
ID: 30, Tag: RBS
ID: 31, Tag: RP
ID: 32, Tag: -RRB-
ID: 33, Tag: SYM
ID: 34, Tag: TO
ID: 35, Tag: UH
ID: 36, Tag: VB
ID: 37, Tag: VBD
ID: 38, Tag: VBG
ID: 39, Tag: VBN
ID: 40, Tag: VBP
ID: 41, Tag: VBZ
ID: 42, Tag: WDT
ID: 43, Tag: WP
ID: 44, Tag: WP$
ID: 45, Tag: WRB


In [12]:
# Load the train, validation, and test datasets
train_iter, test_iter = CoNLL2000Chunking(root='.data', split=('train', 'test'))

In [13]:
# Function to extract unique chunk tags
def extract_tags(data_iter):
    tags = set()
    for sentence in data_iter:
        for word, pos, chunk in sentence:
            tags.add(chunk)
    return tags

# Extract unique chunk tags from the training data
train_tags = extract_tags(train_iter)
print("Unique Chunk Tags:", train_tags)

AttributeError: module 'portalocker' has no attribute 'LockFlags'
This exception is thrown by __iter__ of _MemoryCellIterDataPipe(remember_elements=1000, source_datapipe=_ChildDataPipe)